<a href="https://colab.research.google.com/github/nay-uku/studia/blob/main/diagnostyka-modelMM/dtu_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Temat:</b>
28. Projekt i implementacja algorytmu wyznaczającego optymalne struktury zgodne z modelem MM (2 osoby)


<b>Autorzy:</b>


*   Karol Baranowski WCY19KB2S4
*   Wojciech Grzyb   WCY19KB2S4



Włączenie/Wyłączenie trybu DEBUG:

In [ ]:
# debug = False
debug = True

In [ ]:
#@title Implementacja funkcji
import itertools as it
import math
import numpy as np
import random
import re
import sys

from google.colab import files


np.set_printoptions(threshold=sys.maxsize)


### Implementacja funkcji sprawdzających czy graf jest t-diagnozowalny ###

def check_necessary_conditions(cube, t, debug=False):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  # Weryfikacja warunku 1:
  if not (n_E >= max(t+3,2*t+1)):
    if debug:
      print("WK: warunek 1 nie jest spełniony!")
      print(f"n_E >= max(t+3,2*t+1): {n_E} >= {max(t+3,2*t+1)}")
    return False
  # Weryfikacja warunku 2:
  for i in range(n_E):
    n_adj = len([j for j in range(n_E) if np.count_nonzero(cube[i][j] == 1) > 0])
    if not (n_adj >= t):
      if debug:
        print("WK: warunek 2 nie jest spełniony!")
      return False
  # WK są spełnione
  if debug:
    print("WK są spełnione.")
  return True

def get_comp_tests(cube):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  comp_tests = []  # deklaracjia PSI'
  for e_k in range(n_E):
    for e_i in range(n_E):
      for e_j in range(n_E):
        if cube[e_k, e_i, e_j] == 1:
          comp_test = (e_k, (e_i, e_j))  # odczytanie testu porównawczego
          comp_tests.append(comp_test)  # dodanie odczytanego testu do zbioru testów porównawczych
  return comp_tests

def check_sufficient_condition_1(E, comp_tests, E1, E2, debug=False):
  for psi in comp_tests:
    e_k = psi[0]
    e_i = psi[1][0]
    e_j = psi[1][1]
    set_1 = set(E) - set(E1) - set(E2)
    set_2 = (set(E1) - set(E2)).union(set(E2) - set(E1))
    if e_k in set_1 and e_i in set_1 and e_j in set_2:
      return True
  return False

def check_sufficient_condition_2_3(E, comp_tests, E1, E2, debug=False):
  for psi in comp_tests:
    e_k = psi[0]
    e_i = psi[1][0]
    e_j = psi[1][1]
    set_1 = set(E) - set(E1) - set(E2)
    set_2 = set(E1) - set(E2)
    set_3 = set(E2) - set(E1)
    if e_k in set_1 and e_i in set_2 and e_j in set_2:
      return True
    if e_k in set_1 and e_i in set_3 and e_j in set_3:
      return True
  return False

def check_sufficient_condition(cube, t, debug=False):
  # Ustawienie zmiennych
  n_E = len(cube)  # rząd grafu
  E = [i for i in range(n_E)]  # otagowanie wierzchołków
  # Weryfikacja warunków:
  comp_tests = get_comp_tests(cube)  # testy porównawcze
  all_Et = list(it.combinations(E, t))  # wygenerowanie wszystkich możliwych t-elementowych zbiorów ze zbioru E
  for E1 in all_Et:
    for E2 in all_Et:
      if E1 == E2:
        continue
      if not (check_sufficient_condition_2_3(E, comp_tests, E1, E2, debug=False) or check_sufficient_condition_1(E, comp_tests, E1, E2, debug=False)):
        if debug:
          print("WW: warunek nie jest spełniony!")
        return False
  if debug:
    print("WW są spełnione.")
  return True


### Implementacja funkcji do konwersji tabeli porównań do sześcianu ###

def comp_tests_to_cube(comp_tests):
  # Ustawienie zmiennych
  n_E = max(max(comp_tests))  # wyznaczenie rzędu grafu
  cube = np.zeros((n_E,n_E,n_E), int)  # deklaracja sześcianu
  for comp_test in comp_tests:
    e_k = comp_test[0]-1  # komparator
    e_i = comp_test[1]-1  # element porównawczy 1
    e_j = comp_test[2]-1  # element porównawczy 2
    cube[e_k,e_i,e_j] = 1
    cube[e_k,e_j,e_i] = 1
  return cube


### Implementacja funkcji do weryfikacji czy sześcian jest prawidłowy ###

def is_cube_valid(cube, t, debug=False):
  # Sprawdzenie znaków w strukturze
  if not issubclass(cube.dtype.type, np.integer):
    print(f"Podana struktura ma niedozwolone znaki (nie składa się wyłącznie z liczb).")
    return False
  # Sprawdzenie wymiaru struktury
  if cube.ndim != 3:
    print(f"Podana struktura nie jest trójwymiarowa.")
    return False
  # Sprawdzenie czy struktura jest sześcianem
  if cube.shape[0] != cube.shape[1] != cube.shape[2]:
    print(f"Podana struktura nie jest sześcianem.")
    return False

  # Sprawdzenie czy struktura zawiera poprawnie określone testy porównawcze:
  # Czy dla każdego e_k macierze cube[e_k] jest symetryczna?
  for e_k in range(len(cube)):
    if not np.array_equal(cube[e_k], cube[e_k].T):
      if debug:
        print(f"cube[{e_k}] nie jest symetryczne!")
        print(cube[e_k])
      return False
  # Czy dla każdego testu porównawczego elementy testu porównawczego są względem siebie różne?
  for e_k in range(len(cube)):
    for e_i in range(len(cube)):
      if cube[e_k, e_k, e_i] == 1 or cube[e_k, e_i, e_k] == 1 or cube[e_k, e_i, e_i] == 1:
        if debug:
          print("Elementy testu porównawczego nie są względem siebie różne!")
        return False
  
  # Sprawdzenie WK
  if not check_necessary_conditions(cube, t, debug):
    return False
  # Sprawdzenie WW
  if not check_sufficient_condition(cube, t, debug):
    return False
  # Sześciań jest prawidłowy
  if debug:
    print("Sześcian jest prawidłowy!")
  return True


### Implementacja algorytmu do generowania 'najtańszych' t-diagnozowalnych SOD ###

def visit(u, visited, matrix, stack):
  if not visited[u]: 
    visited[u] = True
    out_neighbours = [v for v in range(len(matrix)) if matrix[u, v] == 1]
    for v in out_neighbours:
      visit(v, visited, matrix, stack)
    stack.append(u)

def assign(u, assigned, matrix, scc):
  if not assigned[u]:
    assigned[u]= True
    out_neighbours = [v for v in range(len(matrix)) if matrix[u, v] == 1]
    for v in out_neighbours:
      scc = assign(v, assigned, matrix, scc)
    scc.append(u)
  return scc

def kosaraju_algorithm(matrix):
  n = len(matrix)
  # Step 1
  visited = [False]*n
  stack = []
  # Step 2
  for u in range(n):
    if not visited[u]:
      visit(u, visited, matrix, stack)
  # Step 3
  matrix_T = matrix.T
  assigned =[False]*(n)
  scc_list = []
  # Step 4
  while stack:
    u = stack.pop()
    if not assigned[u]:
      scc = assign(u, assigned, matrix_T, [])
      scc_list.append(scc)
  return scc_list

def get_scc(tran_matrix):
  scc_list = kosaraju_algorithm(tran_matrix)
  max_scc = scc_list[0]
  for scc in scc_list:
    if len(scc) > len(max_scc):
      max_scc = scc
  return max_scc

def gen_opt_matrix(matrix, t, verify=True):
  # Ustawienie zmiennych
  n = len(matrix)
  gen_matrix = np.zeros((n,n), int)
  # Step 1 - wyznaczenie silnej składowej spójności i dodanie jej do generowanej struktury
  scc = get_scc(matrix)
  i = scc[0]
  for j in range(1, len(scc)):
    gen_matrix[i, scc[j]] = 1
    i = scc[j]
  gen_matrix[i, scc[0]] = 1
  # Step 2 - równomierne dodanie łuków, tak aby warunek konieczny 2 był spełniony
  for i in range(n):
    out_neighbours = [j for j in range(n) if matrix[i, j] == 1 and gen_matrix[i, j] == 0]
    if len(out_neighbours) < t-1:
      raise Exception(f"out_neighbours < t-1; {len(out_neighbours)}<{t-1}")
    if i == 0:
      for j in np.random.permutation(out_neighbours)[:t-1]:
        gen_matrix[i, j] = 1
    else:
      # Wyznaczenie 'najlżejszych' sąsiadów rozpatrywanego węzła
      weights = {}  # wagi poszczególnych sąsiadów
      for j in range(n):
        if j in out_neighbours:
          weights[j] = np.count_nonzero(gen_matrix.T[j])
      sorted_weights = dict(sorted(weights.items(), key=lambda item: item[1]))
      selected_neighbours = dict(it.islice(sorted_weights.items(), t-1))  # wybór 'najlżejszych' sąsiadów
      for j in selected_neighbours:
        gen_matrix[i, j] = 1
  gen_cube = tran_matrix_to_cube(gen_matrix)
  # Step 3 - równomierne dodanie łuków, tak aby struktura była t-diagnozowalna 
  while not is_cube_valid(gen_cube, t):
    # Ustalenie dostępnych (niewykorzystanych) łuków
    available_edges = []
    for i in range(n):
      for j in range(n):
        if matrix[i, j] == 1 and gen_matrix[i, j] == 0:
          available_edges.append((i, j))

    # Wyznaczenie łuku do dodania
    weights = {}  # wagi poszczególnych węzłów 
    for j in range(n):
      weights[j] = np.count_nonzero(gen_matrix.T[j])
    sorted_weights = dict(sorted(weights.items(), key=lambda item: item[1]))
    for j in sorted_weights:
      j_edges = [edge for edge in available_edges if edge[1] == j]
      if len(j_edges) > 0:
        i_weights = {}  # wagi wyznaczonych węzłów
        for edge in j_edges:
          i = edge[0]
          i_weights[i] = np.count_nonzero(gen_matrix[i])
        sorted_i_weights = dict(sorted(i_weights.items(), key=lambda item: item[1]))
        new_edge = (next(iter(sorted_i_weights)), j)
        break
    # Alternatywne wyznaczenie łuku do dodania:
    # new_edge = random.choice(available_edges)

    # Dodanie do generowanej struktury nowego łuku
    gen_matrix[new_edge[0], new_edge[1]] = 1
    gen_cube = tran_matrix_to_cube(gen_matrix)
  # Step 4 - usunięcie (potencjalnie) nadmiarowych łuków
  is_end = False
  while not is_end:
    is_end = True
    edge_list = []  # lista wyznaczonych łuków
    for i in range(n):
      for j in range(n):
        if gen_matrix[i, j] == 1:
          edge_list.append((i, j))
    # Usunięcie nadmiarowych łuków
    for edge in edge_list:
      new_matrix = gen_matrix.copy()
      new_matrix[edge[0], edge[1]] = 0
      new_cube = tran_matrix_to_cube(new_matrix)
      if is_cube_valid(new_cube, t):
        is_end = False
        gen_matrix = new_matrix
        break
  return gen_matrix


### Implementacji funkcji do wypisania (unikalnych) testów porównawczych (do SimDiag-a) ###

def print_uniq_comp_tests(cube):
  uniq_comp_tests = set()
  for test in get_comp_tests(cube):
    uniq_comp_tests.add((test[0]+1, tuple(sorted((test[1][0]+1,test[1][1]+1)))))
  print(f"{'P':<4} {'K':<2} {'u':<2} {'v':<2}")
  for i, test in enumerate(sorted(uniq_comp_tests)):
    print(f"{str(i+1)+'.':<4} {test[0]:<2} {test[1][0]:<2} {test[1][1]:<2}")


### Implementacja funkcji konwertujących macierz przejść w sześciań i na odwrót ###

def tran_matrix_to_cube(tran_matrix):
  # Ustawienie zmiennych
  n = len(tran_matrix)
  E = [i for i in range(n)]  # otagowanie wierzchołków
  n_E = len(E)
  # Określenie zbioru prób porównawczych
  all_Psi = []
  for e in range(n_E):
    all_adj_nodes = list(it.combinations(np.delete(E, e), 2))  # lista możliwych wierzchołków przyległych do komparatora e
    for adj_nodes in all_adj_nodes:
      if tran_matrix[e][adj_nodes[0]] == 1 and tran_matrix[e][adj_nodes[1]] == 1:
        all_Psi.append([e,adj_nodes])
  for i in range(n):
    all_adj_nodes = list(it.combinations(np.delete(E, e), 2))
    for adj_nodes in all_adj_nodes:
      if tran_matrix[e][adj_nodes[0]] == 1 and tran_matrix[e][adj_nodes[1]] == 1:
        all_Psi.append([e,adj_nodes])
  # Utworzenie sześcianu
  cube = np.zeros((n,n,n), int)  # deklaracja sześcianu
  for psi in all_Psi:
    e_k = psi[0]
    e_i = psi[1][0]
    e_j = psi[1][1]
    cube[e_k, e_i, e_j] = 1
    cube[e_k, e_j, e_i] = 1
  return cube

def cube_to_tran_matrix(cube):
  # Ustawienie zmiennych
  n = len(cube)
  tran_matrix = np.zeros((n,n), int)  # deklaracja macierzy przejść
  for k in range(n):
    for i in range(n):
      for j in range(n):
        if cube[k, i, j] == 1:
          tran_matrix[k, i] = 1
          tran_matrix[k, j] = 1
  return tran_matrix


Załadowanie parametrów:

In [ ]:
uploaded = files.upload()

Saving n4_t1_minmax_t.txt to n4_t1_minmax_t.txt
Saving n5_t2_minmax_f.txt to n5_t2_minmax_f.txt
Saving n5_t2_minmax_t.txt to n5_t2_minmax_t.txt
Saving n7_t3_minmax_f.txt to n7_t3_minmax_f.txt
Saving n7_t3_minmax_t.txt to n7_t3_minmax_t.txt
Saving n9_t4_minmax_f.txt to n9_t4_minmax_f.txt
Saving n9_t4_minmax_t.txt to n9_t4_minmax_t.txt
Saving n11_t5_minmax_f.txt to n11_t5_minmax_f.txt
Saving n11_t5_minmax_t.txt to n11_t5_minmax_t.txt
Saving n13_t6_minmax_f.txt to n13_t6_minmax_f.txt
Saving n13_t6_minmax_t.txt to n13_t6_minmax_t.txt
Saving n15_t7_minmax_f.txt to n15_t7_minmax_f.txt
Saving n15_t7_minmax_t.txt to n15_t7_minmax_t.txt
Saving n16_t4_h4_t.txt to n16_t4_h4_t.txt


Realizacja zadania:

In [ ]:
# Załadowanie tabeli porównań
file_name = str(input("Podaj nazwę pliku z tabelą porównań: "))
comp_tests = []
with open(file_name, 'r') as f:
  for line in f:
    pattern = re.compile("^\|(\s+\d+\s+\|){4}$")
    if pattern.match(line):
      comp_test = [int(x) for x in re.findall("\d+", line)[1:]]  # Test porównawczy
      # Weryfikacja testu porównawczego [e_k, e_i, e_j]:
      # Czy e_k, e_i, e_j są względem siebie różne?
      if len(comp_test) != 3 or len(set(comp_test)) < 3:
        raise Exception("Podana tabela porównań jest nieprawidłowa!")
      comp_tests.append(comp_test)
  if len(comp_tests) == 0:
    raise Exception("Nieprawidłowa struktura pliku!")

# Załadowanie parametru t
t = int(input("Podaj parametr t: "))
# Weryfikacja czy podany parametr jest prawidłowy
if max(max(comp_tests)) < max(t+3,2*t+1):
  raise Exception("Nieprawidłowy parametr t!")

# Konwersja tabeli porównań do sześcianu
cube = comp_tests_to_cube(comp_tests)
print("Przekonwertowano wczytaną tabelę do postaci sześcianu.")

# Weryfikacja czy wczytany sześcian jest prawidłowy
if not is_cube_valid(cube, t, debug):
  raise Exception("Wczytana struktura NIE jest prawidłowa!")
print("Weryfikacja sześcianu zakończona pomyślnie.")

# Konwersja sześcianu do macierzy przejść
tran_matrix = cube_to_tran_matrix(cube)
print("Wczytana macierz przejść:")
print(tran_matrix)

# Wygenerowanie (quasi) najtańszej struktury t-diagnozowalnej
gen_matrix = gen_opt_matrix(tran_matrix, t)  # wygenerowana macierz przejść
gen_cube = tran_matrix_to_cube(gen_matrix)  # wygenerowany sześciań
print("Wygenerowana macierz:")
print(gen_matrix)

Podaj nazwę pliku z tabelą porównań: n11_t5_minmax_f.txt
Podaj parametr t: 5
Przekonwertowano wczytaną tabelę do postaci sześcianu.
WK są spełnione.
WW: warunek nie jest spełniony!


Exception: ignored

Weryfikacja czy wygenerowana struktura jest (quasi) najtańsza:

In [ ]:
n = len(gen_matrix)

# Odczytanie wygenerowanych łuków
edge_list = []
for i in range(n):
  for j in range(n):
    if gen_matrix[i, j] == 1:
      edge_list.append((i, j))
# Weryfikacja czy usunięcie jakiegokolwiek łuku spowoduje, że struktura nie będzie już t-diagnozowalna
for edge in edge_list:
  new_matrix = gen_matrix.copy()
  new_matrix[edge[0], edge[1]] = 0
  new_cube = tran_matrix_to_cube(new_matrix)
  if is_cube_valid(new_cube, t):
    raise Exception("Wyznaczona struktura nie jest (quasi) najtańsza!")

print("Wyznaczona struktura jest (quasi) najtańsza.")

Wyznaczona struktura jest (quasi) najtańsza.


Weryfikacja funkcji do sprawdzania t-diagnozowalności na podstawie omówionych na wykładzie przykładów:

In [ ]:
# Przykład z wykładu (slajd 67)
print('Weryfikacja przykładu ze slajdu 67')
matrix_67 = [[0, 1, 0, 1, 1],
             [1, 0, 1, 0, 1],
             [0, 1, 0, 1, 1],
             [1, 0, 1, 0, 0],
             [1, 1, 1, 0, 0]]
t_67 = 2
cube_67 = tran_matrix_to_cube(matrix_67)
if is_cube_valid(cube_67, t_67, debug=True):
  raise Exception("Błąd! Struktura nie jest t-diagnozowalna.")

# Przykład z wykładu (slajd 70)
print('Weryfikacja przykładu ze slajdu 70')
h4 = [[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
      [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
      [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
      [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      [1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
      [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
      [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
      [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
      [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1],
      [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
      [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
      [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
      [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]]
t_h4 = 4
cube_h4 = tran_matrix_to_cube(h4)
if not is_cube_valid(cube_h4, t_h4, debug=True):
  raise Exception("Błąd! Struktura nie powinna być t-diagnozowalna.")
print("Weryfikacja zakończona pomyślnie.")

Weryfikacja przykładu ze slajdu 67
WK są spełnione.
WW: warunek nie jest spełniony!
Weryfikacja przykładu ze slajdu 70
WK są spełnione.
WW są spełnione.
Sześcian jest prawidłowy!
Weryfikacja zakończona pomyślnie.


Implemantacja funkcji Analytic: MinN_MaxT z SimDiag:

In [ ]:
# Implementacja funkcji Analytic: MinN_MaxT z SimDiag
def gen_opt_cube_analytic(t):
  n = max(t+3,2*t+1)
  cube = np.zeros((n,n,n), int)  # deklaracja sześcianu
  # Wygenerowanie t+1 prób porównawczych dla każdego komparatora
  for e_k in range(n):
    e_i = (e_k+1)%n
    for e_j in [e_j for e_j in range(n) if e_j != e_i and e_j != e_k and e_j != (e_k+2)%n ]:
      cube[e_k, e_i, e_j] = 1
      cube[e_k, e_j, e_i] = 1
  return cube

# Wczytanie parametru t
t = int(input("Podaj parametr t: "))

# Wygenerowanie struktury przy użyciu funkcji gen_opt_cube_analytic
gen_cube = gen_opt_cube_analytic(t)
if is_cube_valid(gen_cube, t, debug):
  print(f"Graf jest {t}-diagnozowalny")
  print("Wygenerowany sześcian:")
  print(gen_cube)
else:
  raise Exception("Graf NIE jest t-diagnozowalny!")
print("Wygenerowana macierz przejść:")
print(cube_to_tran_matrix(gen_cube))
print("Wygenerowana tabela porównań:")
print_uniq_comp_tests(gen_cube)

Podaj parametr t: 3
WK są spełnione.
WW są spełnione.
Sześcian jest prawidłowy!
Graf jest 3-diagnozowalny
Wygenerowany sześcian:
[[[0 0 0 0 0 0 0]
  [0 0 0 1 1 1 1]
  [0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0]
  [1 0 0 0 1 1 1]
  [0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 0 0 1 0 0 0]
  [0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0]
  [1 1 0 0 0 1 1]
  [0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 1 0 0]
  [0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0]
  [1 1 1 0 0 0 1]
  [0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 0 0 1 0]
  [0 0 0 0 0 1 0]
  [0 0 0 0 0 1 0]
  [0 0 0 0 0 1 0]
  [0 0 0 0 0 0 0]
  [1 1 1 1 0 0 0]
  [0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1]
  [0 0 0 0 0 0 1]
  [0 0 0 0 0 0 1]
  [0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0]
  [0 1 1 1 1 0 0]]

 [[0 0 1 1 1 1 0]
  [0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0]
  [1 0 0 0 0 0 0]
  [1 0 0 0 0 